## **Setup**

<div class="align-center">
  <a href="https://getindexify.ai/"><img src="https://getindexify.ai/Indexify_Logo_Wordmark.svg" width="145"></a>
  <a href="https://discord.com/invite/kF8UZACA7r"><img src="https://raw.githubusercontent.com/rishiraj/random/main/Discord%20button.png" width="145"></a><br>
  Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/tensorlakeai/indexify">Github</a></i> ⭐
</div>

In [ ]:
%pip install pydantic indexify indexify-extractor-sdk


# Download Indexify Server
!curl https://getindexify.ai | sh

# Download Extractors
!indexify-extractor download tensorlake/instructor

After installing the necessary libraries, download the server, and the extractors, you need to restart the runtime. Then, you have to run Indexify Server with the Extractors.

Open 2 terminals and run the following commands:

```bash
# Terminal 1
./indexify server -d

# Terminal 2
indexify-extractor join-server
```

## **Create Extraction Pipeline**

In [4]:
from typing import List
from pydantic import BaseModel, Field

class Extraction(BaseModel):
    topic: str
    summary: str
    hypothetical_questions: List[str] = Field(
        default_factory=list,
        description="Hypothetical questions that this document could answer",
    )
    keywords: List[str] = Field(
        default_factory=list, description="Keywords that this document is about"
    )


In [5]:
text_chunk = """
## Simple RAG

****What is it?****

The simplest implementation of RAG embeds a user query and do a single embedding search in a vector database, like a vector store of Wikipedia articles. However, this approach often falls short when dealing with complex queries and diverse data sources.

**What are the limitations?**

- **Query-Document Mismatch:** It assumes that the query and document embeddings will align in the vector space, which is often not the case.
    - Query: "Tell me about climate change effects on marine life."
    - Issue: The model might retrieve documents related to general climate change or marine life, missing the specific intersection of both topics.
- **Monolithic Search Backend:** It relies on a single search method and backend, reducing flexibility and the ability to handle multiple data sources.
    - Query: "Latest research in quantum computing."
    - Issue: The model might only search in a general science database, missing out on specialized quantum computing resources.
- **Text Search Limitations:** The model is restricted to simple text queries without the nuances of advanced search features.
    - Query: "what problems did we fix last week"
    - Issue: cannot be answered by a simple text search since documents that contain problem, last week are going to be present at every week.
- **Limited Planning Ability:** It fails to consider additional contextual information that could refine the search results.
    - Query: "Tips for first-time Europe travelers."
    - Issue: The model might provide general travel advice, ignoring the specific context of first-time travelers or European destinations.
"""


In [6]:
from indexify import IndexifyClient
client = IndexifyClient()

In [7]:
client.extractors()

[Extractor(name=tensorlake/instructor, description=Instructor Extractor, input_params={'properties': {'instructions': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'Instructions'}, 'model': {'default': 'gpt-3.5-turbo', 'title': 'Model', 'type': 'string'}, 'schema_bytes': {'format': 'binary', 'title': 'Schema Bytes', 'type': 'string'}, 'system_message': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'default': None, 'title': 'System Message'}}, 'required': ['schema_bytes'], 'title': 'InputParams', 'type': 'object'}, input_mime_types=['text/plain'], outputs={'model': {'metadata': {'age': {'comment': None, 'type': 'int'}, 'name': {'comment': None, 'type': 'text'}, 'place': {'comment': None, 'type': 'text'}}}})]

In [ ]:
import cloudpickle
import base64
import json

extraction_model = base64.b64encode(cloudpickle.dumps(Extraction))
extraction_model = extraction_model.decode('utf-8')

extraction_graph_spec = f"""
name: "instructor"
extraction_policies:
  - extractor: "tensorlake/instructor"
    name: "extract_chunks"
    input_params:
      - schema_bytes: {extraction_model}
      - system_message: "Your role is to extract chunks from the following and create a set of topics."
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)  

In [61]:
content_id = client.add_documents("instructor", text_chunk)
client.wait_for_extraction(content_id)